In [1]:
##########################################
# This notebook is used for intial data exploration for the capstone project
##########################################


First, we import the data

In [1]:
import pandas as pd
import numpy as np
import glob
from IPython.display import display # Allows the use of display() for DataFrames


data = pd.read_csv('data/transactions_200816.csv')  #import one file

#import all csvs (courtesy of http://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

path =r'data' # use your path
allFiles = glob.glob(path + "/tr*.csv")

#print allFiles
#data = pd.DataFrame()
#list_ = []
#for file_ in allFiles:
#    df = pd.read_csv(file_,index_col=None, header=0)
#    list_.append(df)
#data = pd.concat(list_)

print('Before DropNA',len(data))
data=data.dropna()
print('After DropNA',len(data))
#data['CUST_CODE'].head()

#list(data.columns.values)

('Before DropNA', 270664)
('After DropNA', 191470)


Reformat the data into a wide dataset using Prod_code_10 as the lowest level of aggregation.  

In [5]:
#create my target variable
data["CUST_LIFESTAGE_PRICE_SENSITIVITY"] = data["CUST_LIFESTAGE"] + "-" + data["CUST_PRICE_SENSITIVITY"]

#create basketprofiles.  Simplest version will be to sum spend in each category in PROD_CODE_20, keeping the target variable as well
#pivot code from http://stackoverflow.com/questions/41046766/using-and-graphing-the-results-of-a-crosstab-dataframe-in-python

data_cross=data.pivot_table(index='BASKET_ID', columns='PROD_CODE_40', values='SPEND', aggfunc=np.sum, fill_value=0)
data_cross.reset_index(level=['BASKET_ID'], inplace=True)

#create target for the classifier
bybasket=data.groupby(['BASKET_ID'])
targetByBasket=pd.DataFrame(bybasket['CUST_LIFESTAGE_PRICE_SENSITIVITY'].first())
targetByBasket.reset_index(level=['BASKET_ID'], inplace=True)

data_cross = pd.merge(data_cross, targetByBasket, how='inner', on = 'BASKET_ID')

data_cross.head(10)
#list(data_cross.columns.values)

PROD_CODE_40,BASKET_ID,D00001,D00002,D00003,D00004,D00005,D00006,D00007,D00008,D00009,CUST_LIFESTAGE_PRICE_SENSITIVITY
0,994111400168267,0.98,4.23,0.00,0.00,0.00,0.0,0.0,0.0,0.0,OT-UM
1,994111400168287,0.00,4.59,0.30,0.00,0.18,0.0,0.0,0.0,0.0,OT-LA
2,994111400168333,0.00,15.23,0.00,0.00,0.00,0.0,0.0,0.0,0.0,OA-UM
3,994111400168387,1.43,6.30,0.81,0.00,0.06,0.0,0.0,0.0,0.0,OA-MM
4,994111400168388,0.42,1.54,0.00,10.14,0.03,0.0,0.0,0.0,0.0,OA-MM
5,994111400168389,7.24,10.27,1.23,0.00,6.86,0.0,0.0,0.0,0.0,OA-MM
6,994111400168470,2.13,6.81,3.65,0.00,0.00,0.0,0.0,0.0,0.0,OA-LA
7,994111400168495,1.71,8.59,2.15,0.00,0.00,0.0,0.0,0.0,0.0,OA-UM
8,994111400168506,0.00,1.99,0.00,0.00,0.00,0.0,0.0,0.0,0.0,OT-XX
9,994111400168589,0.00,37.15,13.96,0.00,0.07,0.0,0.0,0.0,0.0,OA-UM


# Naive Classifier Work

In [ ]:
#do all the work for the naive classifier

#create variables for the naive classifier
spendByBasket=pd.DataFrame(bybasket['SPEND'].sum())
spendByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketTypeByBasket=pd.DataFrame(bybasket['BASKET_TYPE'].first())
basketTypeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketSizeByBasket=pd.DataFrame(bybasket['BASKET_SIZE'].first())
basketSizeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

weekdayByBasket=pd.DataFrame(bybasket['SHOP_WEEKDAY'].first())
weekdayByBasket.reset_index(level=['BASKET_ID'], inplace=True)

#merge them back in
data_cross_naive = pd.merge(data_cross, spendByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketTypeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketSizeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, weekdayByBasket, how='inner', on = 'BASKET_ID')

In [15]:
#create the naive predictor

avgTrainSpend=X_train['SPEND'].mean()
stDevTrainSpend=X_train['SPEND'].std()


def NaiveClassifier(test, trainMean, trainStd):

    if(test['SPEND']<trainMean-trainStd):
            if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'LA'
            else:
                    naivePriceSens = 'MM'
    elif(test['SPEND']>rainMean-trainStd):
        if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'MM
            else:
                    naivePriceSens = 'UM'
    else:
        naivePriceSens = 'MM'
    
    if (test["SHOP_WEEKDAY"] = 7) or (test["SHOP_WEEKDAY"] = 1):
        if(test['BASKET_SIZE'] = 'Small'):
            if(test['BASKET_TYPE']='Small Shop')
                   naivePriceSens = 'YA'
            else:
                    naivePriceSens = 'OA'
        elif(test['BASKET_SIZE'] = 'Medium'):
            naiveLifeStage = 'YF'
        else:
            naiveLifeStage = 'OF'            
    else:
        naiveLifeStage = 'PE'
    
    
    
    return naivePriceSens + "-" naiveLifeStage

#apply to every row
#X_test.apply(NaiveClassifier, args=(avgTrainSpend,stDevTrainSpend)) # returns DataFrame


In [ ]:
#pop the two classifiers used for the naive predictor before doing any classification
X_train.pop("SPEND")
X_train.pop("SHOP_WEEKDAY")
X_test.pop("SPEND")
X_test.pop("SHOP_WEEKDAY")

# Split the data

In [6]:
#reset the index to what it should be for the rest
data_cross.set_index(['BASKET_ID'], inplace=True)
#create the training, testing split
from sklearn.cross_validation import train_test_split
#split off the target variable
y = data_cross.pop("CUST_LIFESTAGE_PRICE_SENSITIVITY")
X = data_cross

#split the data into training and testing, from http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42) 

# Dimensionality Reduction

In [10]:
print(X_train.describe())

PROD_CODE_40       D00001        D00002        D00003        D00004  \
count         26100.00000  26100.000000  26100.000000  26100.000000   
mean              0.78983      6.365957      2.833409      0.196687   
std               1.50270      8.792910      4.818670      1.574917   
min               0.00000      0.000000      0.000000      0.000000   
25%               0.00000      0.680000      0.000000      0.000000   
50%               0.00000      2.950000      0.930000      0.000000   
75%               1.03000      8.710000      3.600000      0.000000   
max              21.03000    146.680000    112.050000     75.900000   

PROD_CODE_40        D00005        D00006        D00007        D00008  \
count         26100.000000  26100.000000  26100.000000  26100.000000   
mean              1.092362      0.031258      0.010041      1.604405   
std               2.576009      0.475748      0.366295     73.682187   
min               0.000000      0.000000      0.000000      0.000000   


In [7]:
print(X_train.head(10))
from sklearn.decomposition import PCA

pca = PCA(n_components = 6)
pca.fit(X_train)

# TODO: Transform the sample log-data using the PCA fit above
X_train_pca= pca.transform(X_train)
X_test_pca= pca.transform(X_test)

#check cumulative sums
display(np.cumsum(pca.explained_variance_ratio_))


print(X_train_pca)

PROD_CODE_40     D00001  D00002  D00003  D00004  D00005  D00006  D00007  \
BASKET_ID                                                                 
994111400403246    0.00    1.05    0.00    0.00    0.03     0.0     0.0   
994111400493436    0.00    4.25    1.07    0.00    0.00     0.0     0.0   
994111400517014    0.00    2.89    0.00    4.48    0.00     0.0     0.0   
994111400609586    0.00    0.00    0.00    0.00    0.00     0.0     0.0   
994111400683991    0.00    0.86    0.00    0.00    0.00     0.0     0.0   
994111400316123    0.00    7.76    0.00    0.00    0.00     0.0     0.0   
994111400719413    0.00    4.81    3.36    3.36    0.00     0.0     0.0   
994111400809061    0.00    1.81    0.00    0.00    0.00     0.0     0.0   
994111400386079    0.54    0.75    0.89    0.00    1.95     0.0     0.0   
994111400490319    0.00    4.59   13.96    0.00   19.81     0.0     0.0   

PROD_CODE_40     D00008  D00009  
BASKET_ID                        
994111400403246    0.00     0.0

array([ 0.97970046,  0.99510732,  0.99811379,  0.99910193,  0.99954926,
        0.99989593])

[[ 1.60482375  6.0608421   1.04866041 -0.27292112 -0.17101976 -0.37644657]
 [ 1.60466016  2.71607612  1.12576012 -0.66394058 -0.16857009 -0.5742376 ]
 [ 1.60495731  4.32688245  1.63545306 -0.3425026   4.31227475 -0.40655179]
 ..., 
 [ 1.60482475  6.38367363  0.93946567 -0.28366113 -0.17127411 -0.36098224]
 [ 1.60456092  3.70937198  0.05296059 -0.71206845 -0.17404147 -0.55659326]
 [ 1.60478555  6.68817931 -0.36886226 -0.38218992  3.90186221 -0.34772273]]


# try the classifiers

In [8]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100)

rfc = rfc.fit(X_train, y_train)
#rfc_pred = rfc.predict(X_test)
print('non-pca: ', rfc.score(X_test,y_test))

rfc2 = rfc.fit(X_train_pca, y_train)
#rfc_pred = rfc.predict(X_test)
print("pca: ", rfc.score(X_test_pca,y_test))

('non-pca: ', 0.089968976215098237)
('pca: ', 0.085143054119269215)


In [ ]:
#requires sklearn 0.18
#conda update sklearn

from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
ann = ann.fit(X_train, y_train)
ann.score(X_test,y_test)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

nn = KNeighborsClassifier(n_neighbors=1)
nn = nn.fit(X_train,y_train)
#nn_pred = nn.pred(X_test)
nn.score(X_test,y_test)

0.67114788004136505

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100)
ada = ada.fit(X_train, y_train)
ada.score(X_test,y_test)

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
#random useful code snippets
list(data.columns.values)
data['SPEND'].dtype
data_cross.head(10)